# Dataset 2: Creation

The goal is to create a dataset that contains the market share of electric vehicles for each country. We will take the maximal value so far (in most cases, this coincides with the newest). We will consider the European countries from dataset 1 as well as U.S. and China. The dataset will be augmented with a host of characteristics that will later be used as predictor variables.

In [1]:
import pandas as pd
import numpy as np

## Import useful data from dataset 1

In [2]:
df_eu = pd.read_csv('../data/ev_share_europe.csv')
df_eu.head()

,time,country,elec_percent
0,2013,Austria,1.003395
1,2013,Belgium,1.276297
2,2013,Croatia,0.006443
3,2013,Cyprus,1.172058
4,2013,Denmark,0.000000


In [3]:
df_max = df_eu.drop('time',axis=1).groupby('country').max()
df_max.columns = ['max_ev_p']
max_country = df_max.max_ev_p.idxmax()
print(max_country,df_max.loc[max_country].values)
df_max.head()

Norway [40.35865784]


,max_ev_p
country,
Austria,3.795771
Belgium,4.672967
Croatia,0.107844
Cyprus,3.723006
Denmark,0.000000


In [4]:
# Sanity check (3.79 should be the highest value)
df_eu[df_eu.country=='Austria']

,time,country,elec_percent
0,2013,Austria,1.003395
28,2014,Austria,1.191123
56,2015,Austria,1.662784
84,2016,Austria,2.553580
112,2017,Austria,3.795771


## China 

### Alternative 1

* https://en.wikipedia.org/wiki/Electric_car_use_by_country (Passenger plug-in market share of total new car sales between 2013 and 2018 for selected countries and selected regional markets, value for 2018)
    * According to this source, the share was 4.2% in 2018
    * Refers to a blogpost: http://ev-sales.blogspot.com/2019/01/china-december-2018.html
    * Not very reliable!
    
### Alternative 2

* Total
    * https://www.statista.com/statistics/233743/vehicle-sales-in-china/
    * 23.71m passenger vehicles sold in China in 2018
* Electric
    * https://en.wikipedia.org/wiki/New_energy_vehicles_in_China
    * (It refers to https://www.d1ev.com/news/shuju/85937)
    * 1.256m NEV sold in China in 2018
    * **Warning:** According to the definition of the Chinese government, NEVs only include battery electric vehicles (BEV), plug-in hybrid electric vehicles (PHEV) and fuel cell electric vehicles (FCEV). But this number also includes busses, vans and commercial vehicles. This is different to the data on the European market, which only includes sales of passenger cars. Thus, the numbers will be an overestimation of electric passenger cars in China.
* Compute the percentage: 0.05297342893293969

### Conclusion

* Since NEVs (according to the Chinese government) include more vehicles than passenger cars, the number is biased upwards
* So, we take a weighted average

In [5]:
df_max.loc['China','max_ev_p'] = 4.2 * 0.6 + 5.3 * 0.4
df_max.loc['China','max_ev_p']

4.640000000000001

## United States

* Alternative 1
    * https://en.wikipedia.org/wiki/Electric_car_use_by_country
    * EV market share in 2018: 2.1%
* Alternative 2
    * http://www.ev-volumes.com/country/usa/
    * 361000 PEVs sold in 2018
    * https://www.statista.com/statistics/199983/us-vehicle-sales-since-1951/
    * 17213500 (total sales of passenger cars in 2018)
    * Percentage: 0.02097
* The sources agree!

In [6]:
df_max.loc['United States','max_ev_p'] = 361000*100/17213500


## Some European countries

All from https://en.wikipedia.org/wiki/Electric_car_use_by_country

In [7]:
new_data = {
    'Norway': 49.1,
    'Iceland': 19,
    'Denmark':2
}
for k,v in new_data.items():
    df_max.loc[k,'max_ev_p'] = v

In [8]:
df_max = df_max[df_max.max_ev_p != 0]

In [9]:
df_max.to_csv('../data/ev_max_p.csv')

# Country characteristics

In [10]:
df_2 = df_max.copy()

# Due to insufficient data
df_2 = df_2.drop('Liechtenstein',axis=0)

## Same-sex marriages

* Source
    * https://en.wikipedia.org/wiki/Same-sex_marriage
    * Text in the introductory paragraph

In [11]:
ssm = 'Austria,Belgium,Denmark,Finland,France,Germany,Iceland,Ireland,Luxembourg,Malta,Netherlands,Norway,Portugal,Spain, Sweden,United Kingdom,US'.split(',')
print(ssm)
df_2['ssm'] = np.zeros(len(df_2))
for country in df_2.index:
    if country in ssm:
        df_2.loc[country,'ssm'] = 1
    
df_2.head()

['Austria', 'Belgium', 'Denmark', 'Finland', 'France', 'Germany', 'Iceland', 'Ireland', 'Luxembourg', 'Malta', 'Netherlands', 'Norway', 'Portugal', 'Spain', ' Sweden', 'United Kingdom', 'US']


,max_ev_p,ssm
country,,
Austria,3.795771,1.0
Belgium,4.672967,1.0
Croatia,0.107844,0.0
Cyprus,3.723006,0.0
Denmark,2.000000,1.0


In [12]:
# Helper function

def add_column_from_csv(filename, column, df):
    data = pd.read_csv(filename, index_col='country').squeeze()
    df[column] = np.zeros(len(df_2))
    for i in data.index:
        if i in df.index:
            df.loc[i,column] = data[i]
    return data

## Gini index

* Source
    * https://data.worldbank.org/indicator/SI.POV.GINI/
        * Latest Gini index (for those countries that are not in the wikipedia table)
    * https://en.wikipedia.org/wiki/List_of_countries_by_income_equality
        * Table: World Bank Gini

In [13]:
add_column_from_csv('../data/gini.csv','gini',df_2)
df_2.head()

,max_ev_p,ssm,gini
country,,,
Austria,3.795771,1.0,30.5
Belgium,4.672967,1.0,28.1
Croatia,0.107844,0.0,32.2
Cyprus,3.723006,0.0,35.6
Denmark,2.000000,1.0,28.5


In [14]:
df_2[df_2.gini == 0]

,max_ev_p,ssm,gini
country,,,


## GDP (PPP) per capita

* Source
    * https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)_per_capita

In [15]:
add_column_from_csv('../data/ppp_per_capita.csv','ppp',df_2)
df_2.head()

,max_ev_p,ssm,gini,ppp
country,,,,
Austria,3.795771,1.0,30.5,52558.0
Belgium,4.672967,1.0,28.1,47561.0
Croatia,0.107844,0.0,32.2,25264.0
Cyprus,3.723006,0.0,35.6,34504.0
Denmark,2.000000,1.0,28.5,50541.0


In [16]:
df_2[df_2.ppp == 0]

,max_ev_p,ssm,gini,ppp
country,,,,


## Social Progress Index

* Source  
    * https://www.socialprogress.org/
    * https://de.wikipedia.org/wiki/Social_Progress_Index

In [17]:
spi_data = add_column_from_csv('../data/social_progress_index.csv','spi',df_2)
df_2.head()

,max_ev_p,ssm,gini,ppp,spi
country,,,,,
Austria,3.795771,1.0,30.5,52558.0,86.76
Belgium,4.672967,1.0,28.1,47561.0,87.39
Croatia,0.107844,0.0,32.2,25264.0,79.60
Cyprus,3.723006,0.0,35.6,34504.0,82.85
Denmark,2.000000,1.0,28.5,50541.0,89.96


In [18]:
df_2[df_2.spi == 0]

,max_ev_p,ssm,gini,ppp,spi
country,,,,,
Malta,1.746633,1.0,29.4,39534.0,0.0


In [19]:
df_2.loc['Malta','spi'] = spi_data.Italy
df_2.loc['Malta','spi']

86.04

## Representation of nationalist parties in parliament

* Sources
    * https://en.wikipedia.org/wiki/List_of_active_nationalist_parties_in_Europe
    * US: https://en.wikipedia.org/wiki/2016_United_States_presidential_election
        * 46.1

In [20]:
nationalist = pd.read_csv('../data/nationalist_party_europe.csv')
nationalist_tot = nationalist.groupby('country').sum()

In [21]:
nationalist_tot.head()

,vote_percent
country,
Albania,0.28
Armenia,6.60
Austria,26.00
Belgium,24.00
Bulgaria,9.24


In [22]:
df_2['nat_p'] = np.zeros(len(df_2))
for i in df_2.index:
    if i in nationalist_tot.index:
        df_2.loc[i,'nat_p'] = nationalist_tot.loc[i,'vote_percent']

df_2.head()

,max_ev_p,ssm,gini,ppp,spi,nat_p
country,,,,,,
Austria,3.795771,1.0,30.5,52558.0,86.76,26.00
Belgium,4.672967,1.0,28.1,47561.0,87.39,24.00
Croatia,0.107844,0.0,32.2,25264.0,79.60,5.80
Cyprus,3.723006,0.0,35.6,34504.0,82.85,3.71
Denmark,2.000000,1.0,28.5,50541.0,89.96,21.10


In [23]:
# Special case: China
df_2.loc['China','nat_p'] = 50

# Popular vote for Republicans in 2016 election
df_2.loc['United States','nat_p'] = 46.1

In [24]:
df_2.loc[['China','United States'],:]

,max_ev_p,ssm,gini,ppp,spi,nat_p
country,,,,,,
China,4.640000,0.0,38.6,16807.0,64.57,50.0
United States,2.097191,0.0,41.5,59532.0,84.78,46.1


## Write to file

In [25]:
# Reorder columns s.t. target variable is the last one
cols = list(df_2.columns.values)
new_cols = cols[1:] + [cols[0]]
df_2 = df_2[new_cols]

In [26]:
df_2.to_csv('../data/dataset2.csv')